In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/required-documents/sample_submission.csv
/kaggle/input/required-documents/train.csv
/kaggle/input/required-documents/test.csv


In [54]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [3]:
train = pd.read_csv("/kaggle/input/required-documents/train.csv")
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
train.describe()
# good method for detecting outliers

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [5]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [7]:
train_copy = train.copy()
train_copy.drop(['Cabin'],axis = 1,inplace = True)

In [8]:
train.drop(['PassengerId','Name'],axis = 1, inplace = True)
train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [9]:
train.dtypes

HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
dtype: object

In [11]:
home_planet = pd.get_dummies(train['HomePlanet'])
# cryo_sleep = pd.get_dummies(train['CryoSleep'])
# destination = pd.get_dummies(train['Destination'])
# vip = pd.get_dummies(train['VIP'])
train = train.join(home_planet)
# train = train.join(cryo_sleep)
# train = train.join(destination)
# train = train.join(vip)
# #from sklearn.preprocessing import OneHotEncoder

ValueError: columns overlap but no suffix specified: Index(['Earth', 'Europa', 'Mars'], dtype='object')

In [90]:
train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [91]:
home_planet = train.iloc[:,0].values
print(home_planet)
print(home_planet.shape)

['Europa' 'Earth' 'Europa' ... 'Earth' 'Europa' 'Europa']
(8693,)


In [92]:
train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [12]:
train = pd.get_dummies(train,columns = ['HomePlanet','CryoSleep','Destination','VIP'])
train.head()


,Cabin,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Earth,Europa,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,False,0,1,...,0,1,0,1,0,0,0,1,1,0
1,F/0/S,24.0,109.0,9.0,25.0,549.0,44.0,True,1,0,...,1,0,0,1,0,0,0,1,1,0
2,A/0/S,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,1,...,0,1,0,1,0,0,0,1,0,1
3,A/0/S,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,1,...,0,1,0,1,0,0,0,1,1,0
4,F/1/S,16.0,303.0,70.0,151.0,565.0,2.0,True,1,0,...,1,0,0,1,0,0,0,1,1,0


Scaling needs to be done.

<p>Age , room service ,food court, shopping mall, spa,vr deck e scaling yapılacak.</p>

In [16]:
scaler = MinMaxScaler()
train_copy.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [19]:
train.drop(['Cabin'],axis=1,inplace = True)


In [27]:
label_encoder = LabelEncoder()
train['Transported'] = label_encoder.fit_transform(train['Transported'])

In [28]:
scaled = scaler.fit_transform(train)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [31]:
train.dtypes


Age                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Transported                    int64
Earth                          uint8
Europa                         uint8
Mars                           uint8
False                          uint8
True                           uint8
55 Cancri e                    uint8
PSO J318.5-22                  uint8
TRAPPIST-1e                    uint8
HomePlanet_Earth               uint8
HomePlanet_Europa              uint8
HomePlanet_Mars                uint8
CryoSleep_False                uint8
CryoSleep_True                 uint8
Destination_55 Cancri e        uint8
Destination_PSO J318.5-22      uint8
Destination_TRAPPIST-1e        uint8
VIP_False                      uint8
VIP_True                       uint8
dtype: object

In [32]:
scaled

array([[4.93670886e-01, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.03797468e-01, 7.60801284e-03, 3.01881729e-04, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [7.34177215e-01, 3.00132617e-03, 1.19947674e-01, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [3.29113924e-01, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [4.05063291e-01, 0.00000000e+00, 3.51859927e-02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.56962025e-01, 8.79458365e-03, 1.57246839e-01, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [ ]:
# Benim amacım scale işlemi yaparak ML modelinin daha düzgün bir hale gelmesini sağlamak bulduğum bu değerleri datasetimin içerisine atarken problem yaşıyorum.

In [ ]:
train_copy

In [ ]:
train_copy.head()

In [ ]:
# EDA AÇISINDAN PLOTTING işlemleri olabilir onları yapabilirim veya ml algoritmalarını deneyerek devam edebilirim.

In [33]:
# ml algolarına bi giriş yap train test split ayır
train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Earth,Europa,Mars,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,...,0,1,0,1,0,0,0,1,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,1,1,0,0,...,1,0,0,1,0,0,0,1,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,0,1,0,...,0,1,0,1,0,0,0,1,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,1,1,0,0,...,1,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,0,0,1,0,...,0,1,0,1,0,1,0,0,0,1
8689,18.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
8690,26.0,0.0,0.0,1872.0,1.0,0.0,1,1,0,0,...,1,0,0,1,0,0,0,1,1,0
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,0,0,1,0,...,0,1,0,1,0,1,0,0,1,0


In [ ]:
train_copy_to_be_trained = train_copy.drop(['Transported'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(train_copy_to_be_trained,train_copy['Transported'],test_size = 0.2, shuffle= True)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

In [36]:
min_max_scaler = MinMaxScaler(feature_range = (0,1)) 
train.iloc[:,:6] = min_max_scaler.fit_transform(train.iloc[:,0:6].values)

In [37]:
train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Earth,Europa,Mars,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.493671,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,0,...,0,1,0,1,0,0,0,1,1,0
1,0.303797,0.007608,0.000302,0.001064,0.024500,0.001823,1,1,0,0,...,1,0,0,1,0,0,0,1,1,0
2,0.734177,0.003001,0.119948,0.000000,0.299670,0.002030,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
3,0.417722,0.000000,0.043035,0.015793,0.148563,0.007997,0,0,1,0,...,0,1,0,1,0,0,0,1,1,0
4,0.202532,0.021149,0.002348,0.006428,0.025214,0.000083,1,1,0,0,...,1,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.518987,0.000000,0.228726,0.000000,0.073322,0.003066,0,0,1,0,...,0,1,0,1,0,1,0,0,0,1
8689,0.227848,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
8690,0.329114,0.000000,0.000000,0.079687,0.000045,0.000000,1,1,0,0,...,1,0,0,1,0,0,0,1,1,0
8691,0.405063,0.000000,0.035186,0.000000,0.015753,0.134049,0,0,1,0,...,0,1,0,1,0,1,0,0,1,0


<h2> Lets try out some ML algorithms on this dataset </h2>

In [51]:
x_train,x_test,y_train,y_test = train_test_split(train.drop(['Transported'],axis=1),train['Transported'],test_size = 0.2,shuffle = True)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))
clf = svm.SVC(kernel='linear')

6954
1739
6954
1739


In [52]:
x_train.shape

(6954, 24)

In [70]:
clf = SVC()
scoring = "accuracy"
score = cross_val_score(clf, x_train, y_train, n_jobs=1, scoring = scoring)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/s

In [63]:
train.dropna(inplace = True)

In [69]:
train = train[np.isfinite(train).all(1)]

In [71]:
train.isna().sum()

Age                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Transported                  0
Earth                        0
Europa                       0
Mars                         0
False                        0
True                         0
55 Cancri e                  0
PSO J318.5-22                0
TRAPPIST-1e                  0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
CryoSleep_False              0
CryoSleep_True               0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
VIP_False                    0
VIP_True                     0
dtype: int64